In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Load dataset

In [2]:
mnist = input_data.read_data_sets("MNIST_data")

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
train_data = mnist.train.images
train_labels = mnist.train.labels

validation_data = mnist.validation.images
validation_labels = mnist.validation.labels

test_data = mnist.test.images
test_labels = mnist.test.labels

print("train_data", train_data.shape, "train_labels", train_labels.shape)
print("validation_data", validation_data.shape, "validation_labels", validation_labels.shape)
print("test_data", test_data.shape, "test_labels", test_labels.shape)

train_data (55000, 784) train_labels (55000,)
validation_data (5000, 784) validation_labels (5000,)
test_data (10000, 784) test_labels (10000,)


# Create input functions for training, validation and testing

In [4]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=64,
    num_epochs=10,
    shuffle=True
)

validation_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": validation_data},
    y=validation_labels,
    num_epochs=1,
    shuffle=False)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

# Create model function

In [5]:
def my_model_fn(features, labels, mode):
    """Model function for our CNN"""
    
    net = tf.reshape(features['x'], [-1, 28, 28, 1])
    for _ in range(3):
        net = tf.layers.conv2d(
            inputs=net,
            filters=32,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        net = tf.layers.max_pooling2d(
            inputs=net,
            pool_size=[2, 2],
            strides=[2,2]
        )
    net = tf.layers.flatten(net)
    net = tf.layers.dense(inputs=net, units=64)
    logits = tf.layers.dense(inputs=net, units=10)
    
    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:    
        export_outputs = {
            'predictions': tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
     

    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, 
                                           logits=logits)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, 
                                      loss=loss, 
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)

# Create estimator

In [6]:
mnist_estimator = tf.estimator.Estimator(
    model_fn=my_model_fn,
    model_dir="E:\\temp\\mnist_estimator"
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'E:\\temp\\mnist_estimator', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028596B4FC18>, '_tf_random_seed': None, '_log_step_count_steps': 100, '_master': '', '_task_type': 'worker', '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_task_id': 0, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_is_chief': True, '_save_summary_steps': 100, '_service': None}


# Train and evaluate

In [7]:
train_spec = tf.estimator.TrainSpec(
    input_fn=train_input_fn,
    max_steps=20000,
)

validation_spec = tf.estimator.EvalSpec(
    input_fn=validation_input_fn,
    steps=50, throttle_secs=60
)


tf.estimator.train_and_evaluate(mnist_estimator, train_spec, validation_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into E:\temp\mnist_estimator\model.ckpt.
INFO:tensorflow:step = 1, loss = 2.29451
INFO:tensorflow:global_step/sec: 203.92
INFO:tensorflow:step = 101, loss = 2.29998 (0.488 sec)
INFO:tensorflow:global_step/sec: 244.338
INFO:tensorflow:step = 201, loss = 2.28371 (0.411 sec)
INFO:tensorflow:global_step/sec: 254.191
INFO:tensorflow:step = 301, loss = 2.28535 (0.391 sec)
INFO:tensorflow:global_step/sec: 253.781
INFO:tensorflow:step = 401, loss = 2.27648 (0.397 sec)
INFO:tensorflow:global_step/sec: 257.773
INFO:tensorflow:step = 501, loss = 2.28658 (0.386 sec)
INFO:tensorflow:global_step/sec: 251.207
INFO:tensorflow:step = 601, loss = 2.27721 (0.397 sec)
INFO:tensorflow:global_step/sec: 247.992
INFO:

INFO:tensorflow:step = 8101, loss = 0.355714 (0.400 sec)
INFO:tensorflow:global_step/sec: 251.854
INFO:tensorflow:step = 8201, loss = 0.254881 (0.397 sec)
INFO:tensorflow:global_step/sec: 259.05
INFO:tensorflow:step = 8301, loss = 0.194899 (0.387 sec)
INFO:tensorflow:global_step/sec: 267.384
INFO:tensorflow:step = 8401, loss = 0.184008 (0.375 sec)
INFO:tensorflow:global_step/sec: 260.403
INFO:tensorflow:step = 8501, loss = 0.144002 (0.382 sec)
INFO:tensorflow:Saving checkpoints for 8594 into E:\temp\mnist_estimator\model.ckpt.
INFO:tensorflow:Loss for final step: 0.323956.
INFO:tensorflow:Starting evaluation at 2018-01-20-00:18:00
INFO:tensorflow:Restoring parameters from E:\temp\mnist_estimator\model.ckpt-8594
INFO:tensorflow:Evaluation [1/50]
INFO:tensorflow:Evaluation [2/50]
INFO:tensorflow:Evaluation [3/50]
INFO:tensorflow:Evaluation [4/50]
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [6/50]
INFO:tensorflow:Evaluation [7/50]
INFO:tensorflow:Evaluation [8/50]
INFO:te

INFO:tensorflow:step = 14395, loss = 0.080406 (0.461 sec)
INFO:tensorflow:global_step/sec: 171.62
INFO:tensorflow:step = 14495, loss = 0.103363 (0.583 sec)
INFO:tensorflow:global_step/sec: 140.16
INFO:tensorflow:step = 14595, loss = 0.174074 (0.720 sec)
INFO:tensorflow:global_step/sec: 247.217
INFO:tensorflow:step = 14695, loss = 0.136028 (0.400 sec)
INFO:tensorflow:global_step/sec: 255.726
INFO:tensorflow:step = 14795, loss = 0.18929 (0.391 sec)
INFO:tensorflow:global_step/sec: 258.305
INFO:tensorflow:step = 14895, loss = 0.0872859 (0.386 sec)
INFO:tensorflow:global_step/sec: 251.854
INFO:tensorflow:step = 14995, loss = 0.154668 (0.396 sec)
INFO:tensorflow:global_step/sec: 257.711
INFO:tensorflow:step = 15095, loss = 0.300641 (0.388 sec)
INFO:tensorflow:global_step/sec: 243.254
INFO:tensorflow:step = 15195, loss = 0.325257 (0.414 sec)
INFO:tensorflow:global_step/sec: 264.525
INFO:tensorflow:step = 15295, loss = 0.22566 (0.375 sec)
INFO:tensorflow:global_step/sec: 259.167
INFO:tensorfl

INFO:tensorflow:Evaluation [11/50]
INFO:tensorflow:Evaluation [12/50]
INFO:tensorflow:Evaluation [13/50]
INFO:tensorflow:Evaluation [14/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [16/50]
INFO:tensorflow:Evaluation [17/50]
INFO:tensorflow:Evaluation [18/50]
INFO:tensorflow:Evaluation [19/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [21/50]
INFO:tensorflow:Evaluation [22/50]
INFO:tensorflow:Evaluation [23/50]
INFO:tensorflow:Evaluation [24/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [26/50]
INFO:tensorflow:Evaluation [27/50]
INFO:tensorflow:Evaluation [28/50]
INFO:tensorflow:Evaluation [29/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [31/50]
INFO:tensorflow:Evaluation [32/50]
INFO:tensorflow:Evaluation [33/50]
INFO:tensorflow:Evaluation [34/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [36/50]
INFO:tensorflow:Evaluation [37/50]
INFO:tensorflow:Evaluation [38/50]
INFO:tensorflow:Eval

# Perform evaluation on the test set

In [8]:
mnist_estimator.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Starting evaluation at 2018-01-20-00:19:00
INFO:tensorflow:Restoring parameters from E:\temp\mnist_estimator\model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-01-20-00:19:00
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9645, global_step = 20000, loss = 0.120347


{'accuracy': 0.96450001, 'global_step': 20000, 'loss': 0.12034734}

# Getting the result of each prediction

In [9]:
test_results = mnist_estimator.predict(input_fn=test_input_fn)
print(test_results)

<generator object Estimator.predict at 0x000002859726CD00>


In [10]:
import itertools

In [11]:
list(itertools.islice(test_results, 3))

INFO:tensorflow:Restoring parameters from E:\temp\mnist_estimator\model.ckpt-20000


[{'classes': 7,
  'probabilities': array([  3.69987532e-07,   7.86086332e-07,   7.95708329e-05,
           2.29506171e-04,   3.28680727e-09,   7.57216139e-06,
           1.24969664e-12,   9.99198139e-01,   8.28645898e-06,
           4.75684617e-04], dtype=float32)},
 {'classes': 2,
  'probabilities': array([  1.52385724e-06,   9.97432391e-04,   9.98102367e-01,
           8.18418397e-04,   1.72759203e-11,   1.57452159e-05,
           1.02934553e-06,   9.45289541e-11,   6.35835167e-05,
           2.45653706e-11], dtype=float32)},
 {'classes': 1,
  'probabilities': array([  7.63254138e-05,   9.95114326e-01,   4.32886183e-04,
           7.90440594e-04,   1.79769762e-03,   8.63841633e-05,
           2.74113641e-04,   5.44166658e-04,   6.88390224e-04,
           1.95345914e-04], dtype=float32)}]